# BERT experiments

В этой тетрадке описаны наши попытки оценки тональности на предобученной модели [**RuBERT for Sentiment Analysis**](https://huggingface.co/blanchefort/rubert-base-cased-sentiment).

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [3]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
import pandas as pd
import re
from tqdm.notebook import tqdm
import nltk
import datasets

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

В этой тетрадке мы работали только с обучающей выборкой.

In [4]:
aspects = pd.read_csv('train_split_aspects.txt', sep='\t',
                      names=['id_text', 'category', 'tokens', 'start', 'end', 'sentiment'])

In [5]:
texts = pd.read_csv('train_split_reviews.txt', sep='\t',
                    names=['id_text', 'text'])

In [6]:
result = pd.merge(aspects, texts, on="id_text", how="outer")

In [ ]:
result

,id_text,category,tokens,start,end,sentiment,text
0,30808,Whole,ресторане,16,25,neutral,Отмечали в этом ресторане день рождение на пер...
1,30808,Interior,первом этаже,43,55,neutral,Отмечали в этом ресторане день рождение на пер...
2,30808,Whole,руководству ресторана,124,145,positive,Отмечали в этом ресторане день рождение на пер...
3,30808,Service,обслуживающему персоналу,147,171,positive,Отмечали в этом ресторане день рождение на пер...
4,30808,Service,сотрудникам,189,200,positive,Отмечали в этом ресторане день рождение на пер...
...,...,...,...,...,...,...,...
3554,16630,Service,обслуживание,85,97,positive,Уютная и тёплая домашняя обстановка! Милый и о...
3555,16630,Food,Еда,99,102,positive,Уютная и тёплая домашняя обстановка! Милый и о...
3556,16630,Service,персоналу,244,253,positive,Уютная и тёплая домашняя обстановка! Милый и о...
3557,16630,Whole,ресторан,294,302,positive,Уютная и тёплая домашняя обстановка! Милый и о...


Эти функции нужны для выделения *окна* токенов вокруг таргета (аспекта).

In [7]:
def word_index(string, start, end):
    word_re = re.compile(r'\S+')
    start_index = len(word_re.findall(string[:start+1]))
    end_index = len(word_re.findall(string[:end]))
    return start_index, end_index

def get_context(text, start, end, left_win, right_win):
    start_index, end_index = word_index(text, start, end)
    text_split = text.split()
    result = []
    for i in range(left_win):
        if start_index - (left_win - i + 1) > 0:
            result.append(text_split[start_index - (left_win - i + 1)])
    result.append(text[start:end])
    for i in range(right_win):
        if end_index + i < len(text_split):
            result.append(text_split[end_index + i])
    return result

In [8]:
result = result.loc[result['sentiment'] != 'both']

In [9]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

Инициализация моделей.

In [10]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/943 [00:00<?, ?B/s]

In [11]:
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True).to(DEVICE)

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [12]:
@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(DEVICE)
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted

In [13]:
labels = {'neutral': 0, 'positive': 1, 'negative': 2}

Один из лучших полученных нами результатов - на большом окне (12 - левый контекст, 8 - правый). Ниже приведён запуск на этих параметрах.

In [340]:
result['window'] = result.apply(lambda d: ' '.join(get_context(d[6], d[3], d[4], 12, 8)), axis=1)

In [343]:
def accuracy(d):    
    pred = predict(list(d['window']))
    golds = list(d['sentiment'])
    true = 0

    all = len(golds)

    for i, gold in enumerate(golds):
        
        if labels[gold] == pred[i]:
            true += 1

    print('accuracy:', round(true / all, 2))

In [344]:
accuracy(result)

accuracy: 0.66


Ниже представлен код, который использовался для валидации лучших параметров для окна. Результаты получились не слишком интересными: большее окно повышает $Accuracy$. Интересно, что наивысший $Accuracy$. Кроме того, различаются метрики на разных классах. На классе положительной тональности метрики значительно выше ($Accuracy$ доходила и до 0,92 на большом окне), чем на классе отрицательной (на нейтральном классе метрики зачастую совсем низкие).

In [ ]:
def validation(start, end):  
    result['window'] = result.apply(lambda d: ' '.join(get_context(d[6], d[3], d[4], start, end)), axis=1)
    pred = predict(list(result['window']))
    golds = list(result['sentiment'])
    true = 0
    all = len(golds)
    for i, gold in enumerate(golds):
        if labels[gold] == pred[i]:
            true += 1
    return true / all


bounds = []
for i in range(15):
    for k in range(15):
        bounds.append((i, k))


accs = []
for b in tqdm(bounds):
    print(f'validation with start = {b[0]}  and end = {b[1]}')
    acc = validation(b[0], b[1])
    accs.append(acc)
    print('accuracy =', acc)
    print('--------------------\n')